# OpenFL - Decentralized Federated Learning on Public Blockchain Systems
### by Anton Wahrstätter, Sajjad Khan & Davor Svetinovic

## MNIST Dataset simulation


### Contents
* [Experiment](#Deploy-Challenger-Contract-and-Start-Experiment)
* [Visualization](#Visualize-Experiment)
* [Individual Transactions](#Transactions)
* [Contracts and Participants](#Contracts-and-Paricipants-(Latex-format))
* [Gas Costs](#Table-with-Gas-Info-(Latex-format))

### Imports
The imports consist of the following:
* PytorchModel - which is the wrapper for the Convolutional Network
* FLManager - the wrapper for the Manager contract
* FLChallenge - the wrapper for the Challenge contract

In [1]:
from openfl.ml.pytorch_model import PytorchModel
from openfl.contracts import FLManager, FLChallenge

False
True


### Select data set

In [2]:
#DATASET = "cifar-10"
DATASET = "mnist"

### RPC Provider

The RPC provides the connection to the respective blockchain, for example Ethereum or the Robsten testnet.

In [3]:
from openfl.utils import require_env_var

RPC_ENDPOINT = require_env_var("RPC_URL")

Loading environment
/home/wired/dev/openFL-2.0/.env/.env.ganache
Loaded environment: /home/wired/dev/openFL-2.0/.env/.env.ganache


### Configurations

In [4]:
NUMBER_OF_GOOD_CONTRIBUTORS = 4
NUMBER_OF_BAD_CONTRIBUTORS = 1
NUMBER_OF_FREERIDER_CONTRIBUTORS = 1
NUMBER_OF_INACTIVE_CONTRIBUTORS = 0

REWARD = int(1e18) # 1 ETH
MINIMUM_ROUNDS = 3 
MIN_BUY_IN = int(1e18) # 1 ETH
MAX_BUY_IN = int(1.8e18) # 1.8 ETH
STANDARD_BUY_IN = int(1e18) # 1 ETH
EPOCHS = 1 #25
BATCH_SIZE = 32 #128
PUNISHFACTOR = 3
FIRST_ROUND_FEE = 50 # 50% OF MIN DEPOSIT

FORK = True # Fork Chain or communicate directly with RPC

NUMBER_OF_CONTRIBUTERS = NUMBER_OF_GOOD_CONTRIBUTORS      + \
                         NUMBER_OF_BAD_CONTRIBUTORS       + \
                         NUMBER_OF_FREERIDER_CONTRIBUTORS + \
                         NUMBER_OF_INACTIVE_CONTRIBUTORS

In [5]:
# Only for the real-net simulation
# In order to use a non-locally forked blockchain, 
# private keys are required to unlock accounts
from openfl.utils.require_env import require_env_var


if FORK == False:
    from web3 import Web3
    w3 = Web3(Web3.HTTPProvider(RPC_ENDPOINT))
    PRIVKEYS = []
    privKeys = require_env_var("PRIVATE_KEYS").split(':')
    for f in privKeys:
        PRIVKEYS.append(f)

    PRIVKEYS = [w3.eth.account.privateKeyToAccount(i) for i in PRIVKEYS]
else:
    PRIVKEYS = None

### Initialized Deep Learning Model and add Participants

In [6]:
pytorch_model = PytorchModel(DATASET, 
                             NUMBER_OF_GOOD_CONTRIBUTORS, 
                             NUMBER_OF_CONTRIBUTERS, 
                             EPOCHS, 
                             BATCH_SIZE, 
                             STANDARD_BUY_IN,
                             MAX_BUY_IN)


for i in range(NUMBER_OF_BAD_CONTRIBUTORS):
    pytorch_model.add_participant("bad",3)

for i in range(NUMBER_OF_FREERIDER_CONTRIBUTORS):
    pytorch_model.add_participant("freerider",1)
    
for i in range(NUMBER_OF_INACTIVE_CONTRIBUTORS):
    pytorch_model.add_participant("inactive",1)

Data Loaded:
Nr. of images for training: 60,000
Nr. of images for testing:  10,000



/home/wired/dev/openFL-2.0/src/openfl/ml/pytorch_model.py:115: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Pytorch Model created:

Net_MNIST(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc1): Linear(in_features=3136, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=10, bias=True)
)

Participant added: Good User
Participant added: Good User
Participant added: Good User
Participant added: Good User
Participant added: Bad User
Participant added: Freerider User


### Initialize and Deploy Manager Contract

In [7]:
manager = FLManager(pytorch_model, True).init(NUMBER_OF_GOOD_CONTRIBUTORS,
                                              NUMBER_OF_BAD_CONTRIBUTORS,
                                              NUMBER_OF_FREERIDER_CONTRIBUTORS,
                                              NUMBER_OF_INACTIVE_CONTRIBUTORS,
                                              MINIMUM_ROUNDS,
                                              RPC_ENDPOINT,
                                              FORK,
                                              PRIVKEYS)

manager.build_contract()

Connected: True
Client: Ganache/v7.7.3/EthereumJS TestRPC/v7.7.3/ethereum-js
Chain ID: 1337
Latest block: 760
Accounts: ['0x259dD7515eEeF9C26826e383665332F53ec97284', '0x9E6Fbc40C07c3De8E3D40f9dc2003CdCA1A7419e', '0x7d0c94A996917324bb31E74C8cDe8c3eB57e7Eb4']
Default account: <web3._utils.empty.Empty object at 0x760ec95481c0>
New Default account: 0x259dD7515eEeF9C26826e383665332F53ec97284
Connected to Ethereum: True
initiated Ganache-Client @ Block Nr. 760

Total Contributers:       6
Good Contributers:        4 (67%)
Malicious Contributers:   1 (17%)
Freeriding Contributers:  1 (17%)
Inactive Contributers:    0 (0%)
Learning Rounds:          3
-----------------------------------------------------------------------------------
Account initiated @ Address 0x259dD7515eEeF9C26826e38... with 77.2 ETH | FAIR USER
Account initiated @ Address 0x9E6Fbc40C07c3De8E3D40f9... with 93.5 ETH | FAIR USER
Account initiated @ Address 0x7d0c94A996917324bb31E74... with 97.4 ETH | FAIR USER
Account initiat

### Deploy Challenger Contract and Start Experiment

In [9]:
configs = manager.deploy_challenge_contract(MIN_BUY_IN,
                                          MAX_BUY_IN,
                                          REWARD, 
                                          MINIMUM_ROUNDS,
                                          PUNISHFACTOR,
                                          FIRST_ROUND_FEE)

model = FLChallenge(manager, 
                    configs,
                    pytorch_model)


model.simulate(rounds=MINIMUM_ROUNDS)

Starting simulation...
-----------------------------------------------------------------------------------

Model deployed    @ Address 0x138d0E402Af384CeA88f7E84E63FD15c7DdfF5D5 | f893cb...

-----------------------------------------------------------------------------------
Account registered: 0x259dD7515eEeF9... | f893cb... | 1,743,179,840,259,141,376 WEI
-----------------------------------------------------------------------------------
Round 3 starts...

=========================== FEDERATED LEARNING START ==============================
Account training:   0x259dD7515eEeF9... | Epoch  1 | Accuracy  85 % | Loss  14.56
Account testing:    0x259dD7515eEeF9... | Epoch  1 | Accuracy  85 % | Loss 153.95
-----------------------------------------------------------------------------------
Account training:   0x9E6Fbc40C07c3D... | Epoch  1 | Accuracy  86 % | Loss  14.59
Account testing:    0x9E6Fbc40C07c3D... | Epoch  1 | Accuracy  85 % | Loss 149.40
-----------------------------------------

ContractLogicError: ('execution reverted: VM Exception while processing transaction: revert SNR', {'hash': None, 'programCounter': 7302, 'result': '0x08c379a000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000003534e520000000000000000000000000000000000000000000000000000000000', 'reason': 'SNR', 'message': 'revert'})

### Visualize Experiment 

In [ ]:
model.visualize_simulation("experiment/figures")

### Transactions

In [ ]:
print("{:<10} - {:^64} -    Gas Used - {}".format("Function", "Transaction Hash", "Success"))
print("------------------------------------------------------------------------------------------")
for f, txhash in model.txHashes:
    r = model.w3.eth.wait_for_transaction_receipt(txhash)
    if r["status"] == 1:
        success = "✅"
    else:
        success = "FAIL"
    
    gas = r["gasUsed"]
    print("{:<10} - {} - {:>9,.0f} -   {}".format(f, txhash, gas, success))

### Contracts and Paricipants (Latex format)

In [ ]:
print("\\renewcommand{\\arraystretch}{1.3}")
print("\\begin{center}")
print("\\begin{tabular}{ c|c }")

print("\\footnotesize{"+"Contract} &"+" \\footnotesize{Address (Ropsten Testnet)} \\\ ")
print("\\hline")
print("\\footnotesize{"+"Ma-1} & \\footnotesize{"+"{}".format(manager.manager.address)+"} \\\ ")
print("\\footnotesize{"+"Ch-1} & \\footnotesize{"+"{}".format(model.model.address)+"} \\\ ")
for i, p in enumerate(model.pytorch_model.participants[:-1] + \
                           model.pytorch_model.disqualified + \
                           [model.pytorch_model.participants[-1]]):
    print("\\footnotesize{"+"P-{}".format(i+1)+"}  & \\footnotesize{"+"{}".format(p.address)+"} \\\ ")

print("\\end{tabular}")
print("\\end{center}")

### Table with Gas Info (Latex format)

In [ ]:
reg = model.gas_register, "register"
fed = model.gas_feedback, "feedback"
clo = model.gas_close, "settle round"
slo = model.gas_slot, "reserve slot"
wei = model.gas_weights, "provide weights**"
dep = manager.gas_deploy, "deployment"
dep = manager.gas_deploy, "deployment"
ext = model.gas_exit, "exit"

tot  = 0
tot2 = 0

print("\\begin{tabular}{ c|c|c }\n\hline\nFunction & Gas Amount & Gas Costs*\\\ \n\hline")
for i, f in [reg,slo,wei,fed,clo]:
    print("{} & {:,.0f} & {:.5f} ETH \\\ ".format(f, sum(i)/len(i), sum(i)/len(i) * 20e9 / 1e18 ))
    tot += sum(i)/len(i)
    if i != clo[0]:
            tot2 += sum(i)/len(i)
        
print("\hline\n\hline")
print("complete round & {:,.0f} & {:.5f} ETH \\\ ".format(tot, tot * 20e9 / 1e18))
print("\hline\n\end{tabular}")